In [27]:
#!/usr/bin/python
%matplotlib inline

import sys
import pickle
import pandas as pd
sys.path.append("../tools/")
import matplotlib.pyplot as plt
from tester import test_classifier
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.metrics import accuracy_score


### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

<h3> Basic Analysis </h3>

In [28]:
dataset = data_dict

In [29]:
#There are 146 total people in the dataset
len(dataset)

146

In [30]:
#Famous Enron poi
print dataset['LAY KENNETH L']

{'salary': 1072321, 'to_messages': 4273, 'deferral_payments': 202911, 'total_payments': 103559793, 'exercised_stock_options': 34348384, 'bonus': 7000000, 'restricted_stock': 14761694, 'shared_receipt_with_poi': 2411, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 49110078, 'expenses': 99832, 'loan_advances': 81525000, 'from_messages': 36, 'other': 10359729, 'from_this_person_to_poi': 16, 'poi': True, 'director_fees': 'NaN', 'deferred_income': -300000, 'long_term_incentive': 3600000, 'email_address': 'kenneth.lay@enron.com', 'from_poi_to_this_person': 123}


In [31]:
#Compared to average employee
print dataset['METTS MARK']

{'salary': 365788, 'to_messages': 807, 'deferral_payments': 'NaN', 'total_payments': 1061827, 'exercised_stock_options': 'NaN', 'bonus': 600000, 'restricted_stock': 585062, 'shared_receipt_with_poi': 702, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 585062, 'expenses': 94299, 'loan_advances': 'NaN', 'from_messages': 29, 'other': 1740, 'from_this_person_to_poi': 1, 'poi': False, 'director_fees': 'NaN', 'deferred_income': 'NaN', 'long_term_incentive': 'NaN', 'email_address': 'mark.metts@enron.com', 'from_poi_to_this_person': 38}


In [32]:
#As a starting point we will include all of the features in our list
features_list = ['poi', 'bonus', 'deferred_income', 'director_fees',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'other', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages',
'total_payments', 'total_stock_value']

In [33]:
#Here we convert our data to a pandas dataframe, which makes it wasier ot work with
enron_data = pd.DataFrame(dataset)

#Transpose our data to turn names of employees into rows and attributes into columns
enron_data = enron_data.transpose()

In [34]:
# Remove errors, these obviously dont belong in this dataset
dataset.pop('TOTAL', 0)
dataset.pop('THE TRAVEL AGENCY IN THE PARK', 0)

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

In [35]:
print enron_data.columns.values
print "Total Number of Features: ", len(enron_data.columns.values)

['bonus' 'deferral_payments' 'deferred_income' 'director_fees'
 'email_address' 'exercised_stock_options' 'expenses' 'from_messages'
 'from_poi_to_this_person' 'from_this_person_to_poi' 'loan_advances'
 'long_term_incentive' 'other' 'poi' 'restricted_stock'
 'restricted_stock_deferred' 'salary' 'shared_receipt_with_poi'
 'to_messages' 'total_payments' 'total_stock_value']
Total Number of Features:  21


<h4> Counting NaN Values </h4>

In [36]:
columns = []

for i in enron_data.columns.values:
    columns.append(i)


for i in columns:
    print i
    print enron_data[enron_data[i]=="NaN"].count()[i]

bonus
64
deferral_payments
107
deferred_income
97
director_fees
129
email_address
35
exercised_stock_options
44
expenses
51
from_messages
60
from_poi_to_this_person
60
from_this_person_to_poi
60
loan_advances
142
long_term_incentive
80
other
53
poi
0
restricted_stock
36
restricted_stock_deferred
128
salary
51
shared_receipt_with_poi
60
to_messages
60
total_payments
21
total_stock_value
20


In [37]:
#Here we see that the names of people categorized as POI. There are only 18 POIs.
print enron_data[enron_data['poi']==True]['poi']

BELDEN TIMOTHY N        True
BOWEN JR RAYMOND M      True
CALGER CHRISTOPHER F    True
CAUSEY RICHARD A        True
COLWELL WESLEY          True
DELAINEY DAVID W        True
FASTOW ANDREW S         True
GLISAN JR BEN F         True
HANNON KEVIN P          True
HIRKO JOSEPH            True
KOENIG MARK E           True
KOPPER MICHAEL J        True
LAY KENNETH L           True
RICE KENNETH D          True
RIEKER PAULA H          True
SHELBY REX              True
SKILLING JEFFREY K      True
YEAGER F SCOTT          True
Name: poi, dtype: object


In [38]:
#Lets manually look at POIs to see if we can get any insights.
print enron_data[enron_data['poi']==True]

                        bonus deferral_payments deferred_income director_fees  \
BELDEN TIMOTHY N      5249999           2144013        -2334434           NaN   
BOWEN JR RAYMOND M    1350000               NaN            -833           NaN   
CALGER CHRISTOPHER F  1250000               NaN         -262500           NaN   
CAUSEY RICHARD A      1000000               NaN         -235000           NaN   
COLWELL WESLEY        1200000             27610         -144062           NaN   
DELAINEY DAVID W      3000000               NaN             NaN           NaN   
FASTOW ANDREW S       1300000               NaN        -1386055           NaN   
GLISAN JR BEN F        600000               NaN             NaN           NaN   
HANNON KEVIN P        1500000               NaN        -3117011           NaN   
HIRKO JOSEPH              NaN             10259             NaN           NaN   
KOENIG MARK E          700000               NaN             NaN           NaN   
KOPPER MICHAEL J       80000

<h4>Basic Observations: </h4>
<br>
None of the POIs have a director fee. Most of them have a bonus. None of them had restricted_stock_deferred, but going back to our previous cell we see that there were 128 NaN values for that column so this doesnt really tell us much. They seem to have a high shared receipt with POI. They also seem to have fairly high total_payments and total_stock_value

In [39]:
dataset = data_dict

In [40]:
#Create new feature
for k, v in dataset.iteritems():
    if v['bonus'] > 8000000:
        dataset.update({ k: {"Unusually_High_Bonus":"True"}})
#     else:
#         dataset.update({ k : {"Unusually_High_Bonus":"False"}})






In [41]:
print dataset

{'METTS MARK': {'salary': 365788, 'to_messages': 807, 'deferral_payments': 'NaN', 'total_payments': 1061827, 'exercised_stock_options': 'NaN', 'bonus': 600000, 'restricted_stock': 585062, 'shared_receipt_with_poi': 702, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 585062, 'expenses': 94299, 'loan_advances': 'NaN', 'from_messages': 29, 'other': 1740, 'from_this_person_to_poi': 1, 'poi': False, 'director_fees': 'NaN', 'deferred_income': 'NaN', 'long_term_incentive': 'NaN', 'email_address': 'mark.metts@enron.com', 'from_poi_to_this_person': 38}, 'BAXTER JOHN C': {'salary': 267102, 'to_messages': 'NaN', 'deferral_payments': 1295738, 'total_payments': 5634343, 'exercised_stock_options': 6680544, 'bonus': 1200000, 'restricted_stock': 3942714, 'shared_receipt_with_poi': 'NaN', 'restricted_stock_deferred': 'NaN', 'total_stock_value': 10623258, 'expenses': 11200, 'loan_advances': 'NaN', 'from_messages': 'NaN', 'other': 2660303, 'from_this_person_to_poi': 'NaN', 'poi': False, 'direct

<h2> Testing Classifiers</h2>

In [15]:
### Extract features and labels from dataset for local testing
data = featureFormat(dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)


error: key  poi  not present


TypeError: 'NoneType' object is not iterable

In [ ]:
# Example starting point. Try investigating other evaluation techniques!
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
#Naive Bayes Classifier
clf = GaussianNB()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

print accuracy_score(pred, labels_test)

In [ ]:
#SVM classifier, literally exact same as Naive Bayes
clf = svm.SVC()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

print accuracy_score(pred, labels_test)

In [ ]:
#Decision Tree Classifier, Surprisingly much lower than SVM and Naive Bayes
#DTC has different accuracy after running multiple times. NB and SVM never change. 
clf = tree.DecisionTreeClassifier()
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

print accuracy_score(pred, labels_test)

In [228]:
#List of all the features
print enron_data.columns.values

['bonus' 'deferral_payments' 'deferred_income' 'director_fees'
 'email_address' 'exercised_stock_options' 'expenses' 'from_messages'
 'from_poi_to_this_person' 'from_this_person_to_poi' 'loan_advances'
 'long_term_incentive' 'other' 'poi' 'restricted_stock'
 'restricted_stock_deferred' 'salary' 'shared_receipt_with_poi'
 'to_messages' 'total_payments' 'total_stock_value']


<h3> Naive Bayes Classifier</h3>

In [208]:
#First we start with all the features to see our precision and recall
features_list = ['poi', 'bonus', 'deferred_income', 'director_fees',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'other', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages',
'total_payments', 'total_stock_value']


#We have passed the mark for Recall but not precision
test_classifier(clf, dataset, features_list, folds = 1000);

error: key  poi  not present


TypeError: 'NoneType' object is not iterable

In [207]:
#Lets try removing 1 feature to see what happens. Lets pick director_fees
features_list = ['poi', 'bonus', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'other', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages',
'total_payments', 'total_stock_value' ]


#Our Precision went up so we are on the right track
test_classifier(clf, dataset, features_list, folds = 1000);

error: key  poi  not present


TypeError: 'NoneType' object is not iterable

In [70]:
#Lets try removing another feature to see what happens. Lets pick "other" it should increase our score
features_list = ['poi', 'bonus', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages',
'total_payments', 'total_stock_value' ]


#Our precision increased 2%, so close we need less than 1% to reach our mark.
test_classifier(clf, dataset, features_list, folds = 1000);

GaussianNB(priors=None)
	Accuracy: 0.78933	Precision: 0.29374	Recall: 0.41300	F1: 0.34331	F2: 0.38198
	Total predictions: 15000	True positives:  826	False positives: 1986	False negatives: 1174	True negatives: 11014



In [71]:
#Maybe salary will help? Lets find out. 
features_list = ['poi', 'bonus', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'restricted_stock',
 'restricted_stock_deferred',  'shared_receipt_with_poi', 'to_messages',
'total_payments', 'total_stock_value' ]


#Nope. Our precision went down, so we are now going in the wrong direction. lets add salary back.
test_classifier(clf, dataset, features_list, folds = 1000);

GaussianNB(priors=None)
	Accuracy: 0.78587	Precision: 0.28201	Recall: 0.39200	F1: 0.32803	F2: 0.36364
	Total predictions: 15000	True positives:  784	False positives: 1996	False negatives: 1216	True negatives: 11004



In [72]:
#Maybe to_messages? Lets see
features_list = ['poi', 'bonus', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
'total_payments', 'total_stock_value' ]


#Yes! we do achieve a greater than 30% precision
test_classifier(clf, dataset, features_list, folds = 1000);

GaussianNB(priors=None)
	Accuracy: 0.79600	Precision: 0.30741	Recall: 0.42300	F1: 0.35606	F2: 0.39342
	Total predictions: 15000	True positives:  846	False positives: 1906	False negatives: 1154	True negatives: 11094



In [73]:
#Final subset feature list for Naive Bayes
# features_list = ['poi', 'bonus', 'deferred_income',
#  'exercised_stock_options', 'expenses', 'from_messages',
#  'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
#  'long_term_incentive', 'restricted_stock',
#  'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
# 'total_payments', 'total_stock_value' ]

# test_classifier(clf, dataset, features_list, folds = 1000);

GaussianNB(priors=None)
	Accuracy: 0.79600	Precision: 0.30741	Recall: 0.42300	F1: 0.35606	F2: 0.39342
	Total predictions: 15000	True positives:  846	False positives: 1906	False negatives: 1154	True negatives: 11094



<h3>Decision Tree Classifier</h3>

In [75]:
#Running the same feature list for a Decision Tree Classifier, we get much different results
features_list = ['poi', 'bonus', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
'total_payments', 'total_stock_value' ]

#Now, niether our Precision nor recall are above 30%
test_classifier(clf, dataset, features_list, folds = 1000);

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.81013	Precision: 0.28077	Recall: 0.27150	F1: 0.27605	F2: 0.27330
	Total predictions: 15000	True positives:  543	False positives: 1391	False negatives: 1457	True negatives: 11609



In [76]:
#Removing bonus we have recall above 30% and Precision less than 1% away.
features_list = ['poi', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive', 'restricted_stock',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
'total_payments', 'total_stock_value' ]

test_classifier(clf, dataset, features_list, folds = 1000);

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.80767	Precision: 0.29748	Recall: 0.32500	F1: 0.31063	F2: 0.31910
	Total predictions: 15000	True positives:  650	False positives: 1535	False negatives: 1350	True negatives: 11465



In [78]:
#Now Removing restricted stock we are over 30% for both Recall and Precision
features_list = ['poi', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
'total_payments', 'total_stock_value' ]

test_classifier(clf, dataset, features_list, folds = 1000);

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.81660	Precision: 0.33078	Recall: 0.36700	F1: 0.34795	F2: 0.35913
	Total predictions: 15000	True positives:  734	False positives: 1485	False negatives: 1266	True negatives: 11515



In [80]:
#Final Feature list for decision tree
features_list = ['poi', 'deferred_income',
 'exercised_stock_options', 'expenses', 'from_messages',
 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
 'long_term_incentive',
 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
'total_payments', 'total_stock_value' ]


test_classifier(clf, dataset, features_list, folds = 1000);

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.81260	Precision: 0.32002	Recall: 0.36050	F1: 0.33905	F2: 0.35160
	Total predictions: 15000	True positives:  721	False positives: 1532	False negatives: 1279	True negatives: 11468



In [27]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.
dump_classifier_and_data(clf, dataset, features_list)